## Creating an index and populating it with documents using Milvus

Simple example on how to ingest PDF documents, then web pages content into a Milvus VectorStore.

Requirements:
- A Milvus instance, either standalone or cluster.
- Connection credentials to Milvus must be available as environment variables: MILVUS_USERNAME and MILVUS_PASSWORD

### Needed packages and imports

In [1]:
!pip install -q einops==0.7.0 langchain==0.1.9 pypdf==4.0.2 pymilvus==2.3.6 sentence-transformers==2.4.0 --quiet

In [2]:
import requests
import os
from langchain.document_loaders import PyPDFDirectoryLoader, WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Milvus

### Base parameters, the Milvus connection info

In [3]:
# Replace values according to your Milvus deployment
MILVUS_HOST = "vectordb-milvus.milvus.svc.cluster.local"
MILVUS_PORT = 19530
MILVUS_USERNAME = "root"
MILVUS_PASSWORD = "Milvus"
MILVUS_COLLECTION = "milvus_demo_collection"

## Initial index creation and document ingestion

#### Download and load pdfs

In [4]:
product_version = "2.13"
documents = [
    "release_notes",
    "introduction_to_red_hat_openshift_ai",
    "getting_started_with_red_hat_openshift_ai_self-managed",
    "openshift_ai_tutorial_-_fraud_detection_example",
    "developing_a_model",
    "integrating_data_from_amazon_s3",
    "working_on_data_science_projects",
    "serving_models",
    "monitoring_data_science_models",
    "managing_users",
    "managing_resources",
    "installing_and_uninstalling_openshift_ai_self-managed",
    "installing_and_uninstalling_openshift_ai_self-managed_in_a_disconnected_environment",
    "upgrading_openshift_ai_self-managed",
    "upgrading_openshift_ai_self-managed_in_a_disconnected_environment",   
]

pdfs = [f"https://access.redhat.com/documentation/en-us/red_hat_openshift_ai_self-managed/{product_version}/pdf/{doc}/red_hat_openshift_ai_self-managed-{product_version}-{doc}-en-us.pdf" for doc in documents]
pdfs_to_urls = {f"red_hat_openshift_ai_self-managed-{product_version}-{doc}-en-us": f"https://access.redhat.com/documentation/en-us/red_hat_openshift_ai_self-managed/{product_version}/html-single/{doc}/index" for doc in documents}

In [5]:
docs_dir = f"rhoai-doc-{product_version}"

if not os.path.exists(docs_dir):
    os.mkdir(docs_dir)

for pdf in pdfs:
    try:
        response = requests.get(pdf)
    except:
        print(f"Skipped {pdf}")
        continue
    if response.status_code!=200:
        print(f"Skipped {pdf}")
        continue  
    with open(f"{docs_dir}/{pdf.split('/')[-1]}", 'wb') as f:
        f.write(response.content)

Skipped https://access.redhat.com/documentation/en-us/red_hat_openshift_ai_self-managed/2.13/pdf/developing_a_model/red_hat_openshift_ai_self-managed-2.13-developing_a_model-en-us.pdf
Skipped https://access.redhat.com/documentation/en-us/red_hat_openshift_ai_self-managed/2.13/pdf/integrating_data_from_amazon_s3/red_hat_openshift_ai_self-managed-2.13-integrating_data_from_amazon_s3-en-us.pdf
Skipped https://access.redhat.com/documentation/en-us/red_hat_openshift_ai_self-managed/2.13/pdf/monitoring_data_science_models/red_hat_openshift_ai_self-managed-2.13-monitoring_data_science_models-en-us.pdf


In [6]:
pdf_folder_path = f"./rhoai-doc-{product_version}"

pdf_loader = PyPDFDirectoryLoader(pdf_folder_path)
pdf_docs = pdf_loader.load()

/opt/app-root/lib64/python3.11/site-packages/pypdf/_crypt_providers/_cryptography.py:32: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from this module in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import AES, ARC4


#### Inject metadata

In [7]:
from pathlib import Path

for doc in pdf_docs:
    doc.metadata["source"] = pdfs_to_urls[Path(doc.metadata["source"]).stem]

#### Load websites

In [8]:
websites = [
    "https://ai-on-openshift.io/getting-started/openshift/",
    "https://ai-on-openshift.io/getting-started/opendatahub/",
    "https://ai-on-openshift.io/getting-started/openshift-ai/",
    "https://ai-on-openshift.io/odh-rhoai/configuration/",
    "https://ai-on-openshift.io/odh-rhoai/custom-notebooks/",
    "https://ai-on-openshift.io/odh-rhoai/nvidia-gpus/",
    "https://ai-on-openshift.io/odh-rhoai/custom-runtime-triton/",
    "https://ai-on-openshift.io/odh-rhoai/openshift-group-management/",
    "https://ai-on-openshift.io/tools-and-applications/minio/minio/",
    "https://access.redhat.com/articles/7047935",
    "https://access.redhat.com/articles/rhoai-supported-configs",
]

In [9]:
website_loader = WebBaseLoader(websites)
website_docs = website_loader.load()

#### Merge both types of docs

In [10]:
docs = pdf_docs + website_docs

#### Split documents into chunks with some overlap

In [11]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024,
                                               chunk_overlap=128)
all_splits = text_splitter.split_documents(docs)
all_splits[0]

Document(page_content='Red Hat OpenShift AI Self-Managed\n2.13\nUpgrading OpenShift AI Self-Managed in a\ndisconnected environment\nUpgrade Red Hat OpenShift AI on OpenShift in a disconnected environment\nLast Updated: 2024-11-07', metadata={'source': 'https://access.redhat.com/documentation/en-us/red_hat_openshift_ai_self-managed/2.13/html-single/upgrading_openshift_ai_self-managed_in_a_disconnected_environment/index', 'page': 0})

#### Create the index and ingest the documents

In [12]:
# If you don't want to use a GPU, you can remove the 'device': 'cuda' argument
model_kwargs = {'device': 'cuda'}
embeddings = HuggingFaceEmbeddings(
    model_kwargs=model_kwargs,
    show_progress=True
)

# BEWARE: `drop_old` is set to True, so if the collection already existed it will deleted first.
db = Milvus(
    embedding_function=embeddings,
    connection_args={"host": MILVUS_HOST, "port": MILVUS_PORT, "user": MILVUS_USERNAME, "password": MILVUS_PASSWORD},
    collection_name=MILVUS_COLLECTION,
    metadata_field="metadata",
    text_field="page_content",
    auto_id=True,
    drop_old=True
    )

In [13]:
db.add_documents(all_splits)

Batches:   0%|          | 0/36 [00:00<?, ?it/s]

[457614308037629814,
 457614308037629815,
 457614308037629816,
 457614308037629817,
 457614308037629818,
 457614308037629819,
 457614308037629820,
 457614308037629821,
 457614308037629822,
 457614308037629823,
 457614308037629824,
 457614308037629825,
 457614308037629826,
 457614308037629827,
 457614308037629828,
 457614308037629829,
 457614308037629830,
 457614308037629831,
 457614308037629832,
 457614308037629833,
 457614308037629834,
 457614308037629835,
 457614308037629836,
 457614308037629837,
 457614308037629838,
 457614308037629839,
 457614308037629840,
 457614308037629841,
 457614308037629842,
 457614308037629843,
 457614308037629844,
 457614308037629845,
 457614308037629846,
 457614308037629847,
 457614308037629848,
 457614308037629849,
 457614308037629850,
 457614308037629851,
 457614308037629852,
 457614308037629853,
 457614308037629854,
 457614308037629855,
 457614308037629856,
 457614308037629857,
 457614308037629858,
 457614308037629859,
 457614308037629860,
 457614308037

#### Alternatively, add new documents

In [14]:
# If you don't want to use a GPU, you can remove the 'device': 'cuda' argument
# model_kwargs = {'device': 'cuda'}
# embeddings = HuggingFaceEmbeddings(
#     model_kwargs=model_kwargs,
#     show_progress=True
# )

# db = Milvus(
#     embedding_function=embeddings,
#     connection_args={"host": MILVUS_HOST, "port": MILVUS_PORT, "user": MILVUS_USERNAME, "password": MILVUS_PASSWORD},
#     collection_name=MILVUS_COLLECTION,
#     metadata_field="metadata",
#     text_field="page_content",
#     auto_id=True,
#     drop_old=False
#     )

# db.add_documents(all_splits)

#### Test query

In [15]:
query = "How can I work with GPU and taints in OpenShift AI?"
docs_with_score = db.similarity_search_with_score(query)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [16]:
for doc, score in docs_with_score:
    print("-" * 80)
    print("Score: ", score)
    print(doc.page_content)
    print("-" * 80)

--------------------------------------------------------------------------------
Score:  0.5976206660270691
Apply the taints you need to your Nodes or MachineSets, for example:
apiVersion: machine.openshift.io/v1beta1
kind: MachineSet
metadata:
  ...
spec:
  replicas: 1
  selector:
    ...
  template:
    ...
    spec:
      ...
      taints:
        - key: restrictedaccess
          value: "yes"
          effect: NoSchedule



Apply the relevant toleration to the NVIDIA Operator.


In the nvidia-gpu-operator namespace, get to the Installed Operator menu, open the NVIDIA GPU Operator settings, get to the ClusterPolicy tab, and edit the ClusterPolicy.
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Score:  0.65267014503479
But don't worry, OpenShift AI and Open Data Hub take care of this part for you when you launch notebooks, workbenches, model servers, or pipeline runtimes